In [75]:
import pandas as pd
import plotly.express as px
import numpy as np

### Import Data

In [ ]:
sales = pd.read_csv('https://storage.googleapis.com/courses_data/Assignment%20CSV/finance_liquor_sales.csv')

### Preparation

In [ ]:
# Convert 'date' to datetime format
sales['date'] = pd.to_datetime(sales['date'])

# Convert 'zip_code' to string
sales['zip_code'] = sales['zip_code'].astype(int).astype(str)

# Date Selection
sales = sales.loc[(sales['date'] > '2016') & (sales['date'] < '2020')]

<hr>

### Task 1 (Query): Most popular items by zip code

In [222]:
# Group by zip_code and item_number, summing bottles_sold
sales_per_item = sales.groupby(['zip_code', 'item_number'])[['bottles_sold', 'item_description']].sum().reset_index()

# Identify the item with the maximum bottles_sold for each zip_code
most_popular_items = sales_per_item.loc[sales_per_item.groupby('zip_code')['bottles_sold'].idxmax()].reset_index()

# Select Top-10 items sorted by bottles_sold column
most_popular_items_sorted = most_popular_items.sort_values('bottles_sold', ascending=False).head(10).copy()
most_popular_items_sorted

,index,zip_code,item_number,bottles_sold,item_description
37,62,52314,75087,1560,Juarez Gold DssJuarez Gold Dss
19,30,50702,77487,768,Tortilla Gold Dss
0,0,50010,946574,288,Member's Mark Spiced Rum
8,13,50314,86251,240,Juarez Triple SecJuarez Triple Sec
23,39,51106,67527,240,Kahlua Coffee Liqueur
39,65,52402,86390,216,Montezuma Triple Sec
20,33,50703,168,180,Pinnacle Peach w/ Punch Dispenser
11,19,50320,973627,120,Di Amore Quattro Orange
15,24,50501,38176,108,Titos Vodka
2,2,50111,77805,108,Saints N Sinners Apple Pie


### Task 1 (Plot): Most popular items by zip code

In [226]:
# Define a trimmed green gradient
greens_trimmed = px.colors.sample_colorscale(px.colors.sequential.Greens,
                                             np.linspace(0.3, 0.9, 6))

# Create bar chart with zip_code on x-axis and bottles_sold on y-axis
fig = px.bar(
    most_popular_items_sorted,
    x="zip_code",
    y="bottles_sold",
    color="bottles_sold",
    color_continuous_scale=greens_trimmed,
    text="bottles_sold",
)

# Format labels above bars: bold, grey color, thousand separator
# Customize hover to show item_description, zip code, and bottles sold
fig.update_traces(
    texttemplate="<i>%{text:,}</i>",
    textposition="outside",
    textfont=dict(color="grey", size=16),
    hovertemplate="%{customdata[0]}"
    ,
    customdata=most_popular_items_sorted[["item_description"]],
    cliponaxis=False
)

# Update layout: title, axes labels, font, background, margins
# Hide y-axis tick labels and remove color scale legend
fig.update_layout(
    title=dict(
        text="Top 10 Best-Selling Items by Zip Code (2016 - 2019)",
        x=0.5,
        font=dict(family="Calibri", size=28, color="black")
    ),
    xaxis_title=dict(text="Zip Code", font=dict(family="Calibri", size=20)),
    yaxis_title="Bottles Sold",
    yaxis=dict(showticklabels=False),
    font=dict(family="Calibri", size=16),
    plot_bgcolor="white",
    coloraxis_showscale=False,
    margin=dict(l=40, r=40, t=70, b=40)
)

# Display figure
fig.show()

<hr>

### Task 2 (Query): Top 10 Stores by Share of Total Sales

In [230]:
# Group sales by store_name and sum sale_dollars
sales_by_store = sales.groupby(['store_name'])['sale_dollars'].sum().reset_index().sort_values(by=['sale_dollars'], ascending=False).reset_index(drop=True)

# Calculate total sales amount
total_sales_amount = sales_by_store['sale_dollars'].sum()

# Calculate sales percentage for each store
sales_by_store['sale_dollars'] = (sales_by_store['sale_dollars'] / total_sales_amount)

# Rename the column from 'sale_dollars' to 'sales_percentage'
sales_by_store = sales_by_store.rename(columns={'sale_dollars': 'sales_percentage'})

# Select Top-10 stores by sales percentage
top_ten_stores = sales_by_store.head(10).sort_values("sales_percentage", ascending=False).copy()
top_ten_stores

,store_name,sales_percentage
0,Wilkie Liquors,0.182035
1,Sam's Club 6432 / Sioux City,0.104029
2,Sam's Club 6514 / Waterloo,0.081446
3,Hy-Vee #3 / BDI / Des Moines,0.064601
4,Sam's Club 6568 / Ames,0.061310
5,Cedar Ridge Vineyards,0.058155
6,Hy-Vee Wine and Spirits / WDM,0.052826
7,Central City 2,0.040424
8,Fareway Stores #138 / Pleasant Hill,0.035950
9,Hy-Vee Food Store #3 / Waterloo,0.035339


### Task 2 (Plot): Top 10 Stores by Share of Total Sales

In [229]:
# Sort descending by sales_percentage
df_sorted = top_ten_stores.sort_values("sales_percentage", ascending=False)
df_sorted["total_sales_calc"] = df_sorted["sales_percentage"] * total_sales_amount


# Trimmed green gradient
blues_trimmed = px.colors.sample_colorscale(
    px.colors.sequential.Blues,
    np.linspace(0.3, 0.9, 6)
)

fig = px.bar(
    df_sorted,
    x="sales_percentage",
    y="store_name",
    orientation="h",
    text="sales_percentage",
    color="sales_percentage",
    color_continuous_scale=blues_trimmed,
    title="Top 10 Stores by Share of Total Sales (2016 - 2019)"
)

# Labels above bars
fig.update_traces(
    texttemplate="<i>%{text:.2%}</i>",  # italic, 2 decimals as percentage
    textposition="outside",
    textfont=dict(color="grey", size=16),
    cliponaxis=False,
    customdata=df_sorted["total_sales_calc"],
    hovertemplate=(
        "Total Sales → $%{customdata:,.0f}<extra></extra>"
    )
)

# Layout styling
fig.update_layout(
    width=750,
    height=600,
    xaxis_title=None,
    yaxis_title=None,
    xaxis=dict(showticklabels=False),
    yaxis=dict(
        categoryorder="total ascending",
        tickfont=dict(size=16, color="black"),
        ticklabelstandoff=10
    ),
    font=dict(family="Calibri", size=16, color="black"),
    plot_bgcolor="white",
    coloraxis_showscale=False,
    title=dict(
        x=0.5,
        font=dict(family="Calibri", size=28, color="black"),
    )
)

fig.show()